# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="https://github.com/maab2198/applied_ds_python/blob/course2/applied_plotting/readonly/Assignment3Fig1.png?raw=1" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="https://github.com/maab2198/applied_ds_python/blob/course2/applied_plotting/readonly/Assignment3Fig2c.png?raw=1" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [2]:
# Use the following data for this assignment:
# %matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.cm as cm
import matplotlib.colors as colors

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3610,3611,3612,3613,3614,3615,3616,3617,3618,3619,3620,3621,3622,3623,3624,3625,3626,3627,3628,3629,3630,3631,3632,3633,3634,3635,3636,3637,3638,3639,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649
1992,-8941.531897,127788.667612,-71887.743011,-79146.060869,425156.114501,310681.166595,50581.575349,88349.230566,185804.513522,281286.947277,233437.871517,-227244.221822,86998.326686,77782.575787,302583.367033,209285.868118,-368327.461932,-42368.507428,365805.061905,-55713.947167,-75948.289104,127397.002082,681788.783886,-172245.504871,-83417.460608,56824.255135,92522.712383,136754.413630,32188.055555,300761.958723,-110708.797019,-134230.707771,-442046.330791,-340152.157710,-140151.479686,144029.058606,-221186.898339,55965.424932,-180702.489611,98576.543122,...,243603.480641,-302456.661122,-141585.235712,-91588.789248,152792.947836,-99992.746357,-134657.373636,27411.244360,-50110.698710,173360.910161,166069.066752,-94092.979208,-8053.847696,-76933.464704,14538.069320,251103.033929,-22733.673307,267716.743708,-395209.384721,271852.854117,120715.697368,-355947.615528,-5307.186400,-473897.771965,404229.924267,376944.510876,122125.022554,-226859.186251,35573.176407,-342424.249551,171938.760289,150650.759924,203663.976475,-377877.158072,-197214.093861,24185.008589,-56826.729535,-67319.766489,113377.299342,-4494.878538
1993,-51896.094813,198350.518755,-123518.252821,-129916.759685,216119.147314,49845.883728,149135.648505,62807.672113,23365.577348,-109686.264981,143973.220873,149211.328778,97627.497748,-47123.013558,189071.044852,116617.008245,19673.406721,99051.501841,28371.886237,31873.611779,-41129.903198,148282.489943,-75385.183578,-17940.948483,5989.008121,12514.078074,51109.775598,50881.532197,149406.296856,31162.466177,30005.538908,-34700.341831,-2066.208534,3938.838730,-30049.318662,-7587.984642,125584.101289,135760.854891,-42737.930841,289556.729096,...,-25063.762799,-6381.525350,-20792.960572,56171.252222,52181.916421,29898.563068,60860.542533,6756.260183,97920.106009,-109317.543905,118097.037989,-86959.395276,-17977.451815,-196871.770735,-78628.715111,113147.009583,-89183.720598,-50979.449596,-65535.636196,-59353.767451,72899.992967,23871.029181,-171961.653041,198285.454409,83507.420667,133038.005845,85680.351811,49043.404885,-153490.820097,200030.512819,-44566.520071,101032.122475,117648.199945,160475.622607,-13759.888342,-37333.493572,103019.841174,179746.127403,13455.493990,34442.898855
1994,152336.932066,192947.128056,389950.263156,-93006.152024,100818.575896,5529.230706,-32989.370488,223942.967178,-66721.580898,47826.269111,-192967.663433,204584.561046,38869.703622,32040.564523,-147185.190711,4025.232730,79921.616963,328776.090270,-182624.859166,-36343.057416,-79062.500439,251626.386131,-78444.334782,203774.943031,15047.966705,320531.493015,304706.467877,224425.509343,-123087.581536,94658.527746,205722.744144,-112143.902641,16981.408505,-61737.300147,-17137.926667,67299.909803,-5186.145961,29057.993918,111944.812978,153957.428589,...,116455.495993,-241373.894471,164776.897349,182004.190861,-93780.340180,-1312.951962,85609.682019,142458.907175,33934.130004,136771.105318,91705.488445,251117.848685,167640.502317,-237201.424606,-251474.769728,233536.303598,75467.603022,-38136.030588,48491.810886,-49579.178708,-169640.335102,135703.619226,106938.598353,139392.240143,-64356.362713,48815.619937,-146941.987837,147184.086629,81242.824039,-65784.913980,165085.806360,74735.174090,107329.726875,199250.734156,-36792.202754,-71861.846997,26375.113219,-29328.078384,65858.761714,-91542.001049
1995,-69708.439062,-13289.977022,-30178.390991,55052.181256,152883.621657,12930.835194,63700.461932,64148.489835,-29316.268556,59645.677367,-52635.307654,34336.438541,35651.001159,5400.939983,100867.454146,54343.043728,-3899.503204,48050.133302,19412.920262,120522.850238,78198.485628,112562.785435,27994.669028,-16385.665394,58009.02837

calculate mean m and margin of error yerr for each of the rows in the data frame
PS. yerr =the margin of error

where margin of error = standard error* C (C is a constant determined by the 95% (Critical value or T-value of 95% of a normal distribution) i.e.abs(qnorm( (1-.95)/2 percentile)), for a normal distribution C is approximately 1.96

and the standard error of the sampling distribution is std_sample/sqrt(Number of samples)


In [0]:
def mean_confidence_interval(data, confidence=0.95):
    n = data.shape[1]
    m = data.mean(axis=1)
    se = data.std(axis = 1)/np.sqrt(n)
    h = se * stats.norm.ppf((1 - confidence) / 2.)
    conf_ints = [stats.norm.interval(confidence, loc=mu, scale=SE) for mu, SE in zip(m, se)]
    return m,h, conf_ints

def bar_color(y,heigh):
    if y < heigh:
        return "red"
    elif y > heigh:
        return "blue"
    else:
        return "white"
    
def prob_over_y(y, c_interval):
    if y < np.min(c_interval):
        prob = 1.0
    elif y > np.max(c_interval):
        prob = 0.0
    else:
        prob = (np.max(c_interval) - y) / (np.max(c_interval) - np.min(c_interval))
    return prob

def onclick(event):
    plt.cla()
    y = event.ydata
    plt.gca().axhline(y = y , color = "grey", alpha = 1)
    probs = [prob_over_y(y, c_interval) for c_interval in intervals]
    plt.bar(range(df.shape[0]), mean, yerr= yerr, color=cpick.to_rgba(probs))
    plt.show()

In [28]:
mean, yerr,intervals = mean_confidence_interval(df)

    
cmap = cm.get_cmap('coolwarm')
cpick = cm.ScalarMappable(cmap=cmap, norm=colors.Normalize(vmin=0, vmax=1.0))
cpick.set_array([])

plt.figure()

threshold=42000
probs = [prob_over_y(threshold, c_interval) for c_interval in intervals]
plt.bar(range(df.shape[0]), mean, yerr= yerr, color=cpick.to_rgba(probs))



plt.axhline(y = threshold, color = "grey", alpha = 1)

plt.xticks(range(df.shape[0]), ['1992', '1993', '1994', '1995'], alpha = 0.8)
plt.title('Ferreira et al, 2014')
cbar = plt.colorbar(cpick, orientation="horizontal")


plt.gcf().canvas.mpl_connect('button_press_event', onclick)


<IPython.core.display.Javascript object>